[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wasim/Data-Science/blob/main/data-analyst-roadmap/06_sql_for_analytics/04_ctes_and_complex_joins.ipynb)

# CTEs and Complex Joins

Master advanced SQL joining techniques.

## Topics Covered
- Common Table Expressions (CTEs)
- Recursive CTEs
- Self joins
- Multiple table joins
- LEFT/RIGHT/FULL OUTER joins
- Cross joins

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')

## Setup: Create Multiple Tables

In [ ]:
cursor = conn.cursor()

# Employees table
cursor.execute('''
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name TEXT,
    dept_id INTEGER,
    manager_id INTEGER,
    salary INTEGER
)
''')

# Departments table
cursor.execute('''
CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT,
    location TEXT
)
''')

# Projects table
cursor.execute('''
CREATE TABLE projects (
    project_id INTEGER PRIMARY KEY,
    project_name TEXT,
    dept_id INTEGER,
    budget INTEGER
)
''')

# Insert data
employees = [
    (1, 'Alice', 1, None, 95000),
    (2, 'Bob', 1, 1, 85000),
    (3, 'Charlie', 2, None, 80000),
    (4, 'David', 2, 3, 70000),
    (5, 'Eve', 3, None, 75000),
    (6, 'Frank', None, None, 60000)
]

departments = [
    (1, 'Engineering', 'Building A'),
    (2, 'Marketing', 'Building B'),
    (3, 'HR', 'Building C'),
    (4, 'Finance', 'Building D')
]

projects = [
    (1, 'Website Redesign', 1, 50000),
    (2, 'Mobile App', 1, 80000),
    (3, 'Ad Campaign', 2, 30000),
    (4, 'Recruitment', 3, 20000)
]

cursor.executemany(
    'INSERT INTO employees VALUES (?,?,?,?,?)', 
    employees
)
cursor.executemany(
    'INSERT INTO departments VALUES (?,?,?)', 
    departments
)
cursor.executemany(
    'INSERT INTO projects VALUES (?,?,?,?)', 
    projects
)
conn.commit()

## 1. Common Table Expressions (CTEs)

CTEs create temporary named result sets.

In [ ]:
# Simple CTE
query = '''
WITH high_earners AS (
    SELECT 
        emp_id,
        name,
        salary
    FROM employees
    WHERE salary > 75000
)
SELECT 
    name,
    salary
FROM high_earners
ORDER BY salary DESC
'''

pd.read_sql_query(query, conn)

In [ ]:
# Multiple CTEs
query = '''
WITH 
dept_stats AS (
    SELECT 
        dept_id,
        COUNT(*) AS emp_count,
        AVG(salary) AS avg_salary
    FROM employees
    WHERE dept_id IS NOT NULL
    GROUP BY dept_id
),
project_stats AS (
    SELECT 
        dept_id,
        COUNT(*) AS project_count,
        SUM(budget) AS total_budget
    FROM projects
    GROUP BY dept_id
)
SELECT 
    d.dept_name,
    ds.emp_count,
    ds.avg_salary,
    ps.project_count,
    ps.total_budget
FROM departments d
LEFT JOIN dept_stats ds 
    ON d.dept_id = ds.dept_id
LEFT JOIN project_stats ps 
    ON d.dept_id = ps.dept_id
ORDER BY d.dept_name
'''

pd.read_sql_query(query, conn)

## 2. Recursive CTEs

For hierarchical data (org charts, etc.).

In [ ]:
# Build org hierarchy
query = '''
WITH RECURSIVE org_chart AS (
    -- Base case: top-level managers
    SELECT 
        emp_id,
        name,
        manager_id,
        1 AS level,
        name AS path
    FROM employees
    WHERE manager_id IS NULL
    
    UNION ALL
    
    -- Recursive case: employees with managers
    SELECT 
        e.emp_id,
        e.name,
        e.manager_id,
        oc.level + 1,
        oc.path || ' -> ' || e.name
    FROM employees e
    JOIN org_chart oc 
        ON e.manager_id = oc.emp_id
)
SELECT 
    emp_id,
    name,
    level,
    path
FROM org_chart
ORDER BY level, name
'''

pd.read_sql_query(query, conn)

## 3. Self Joins

Join a table to itself.

In [ ]:
# Find employee-manager pairs
query = '''
SELECT 
    e.name AS employee,
    e.salary AS emp_salary,
    m.name AS manager,
    m.salary AS mgr_salary
FROM employees e
LEFT JOIN employees m 
    ON e.manager_id = m.emp_id
ORDER BY e.name
'''

pd.read_sql_query(query, conn)

In [ ]:
# Find employees in same department
query = '''
SELECT DISTINCT
    e1.name AS employee1,
    e2.name AS employee2,
    e1.dept_id AS department
FROM employees e1
JOIN employees e2 
    ON e1.dept_id = e2.dept_id
    AND e1.emp_id < e2.emp_id
WHERE e1.dept_id IS NOT NULL
ORDER BY e1.dept_id, e1.name
'''

pd.read_sql_query(query, conn)

## 4. INNER vs OUTER Joins

In [ ]:
# INNER JOIN - only matching rows
query = '''
SELECT 
    e.name,
    d.dept_name
FROM employees e
INNER JOIN departments d 
    ON e.dept_id = d.dept_id
ORDER BY e.name
'''

print("INNER JOIN (only employees with dept):")
display(pd.read_sql_query(query, conn))

In [ ]:
# LEFT JOIN - all from left table
query = '''
SELECT 
    e.name,
    d.dept_name
FROM employees e
LEFT JOIN departments d 
    ON e.dept_id = d.dept_id
ORDER BY e.name
'''

print("LEFT JOIN (all employees):")
display(pd.read_sql_query(query, conn))

In [ ]:
# RIGHT JOIN (simulated with LEFT)
query = '''
SELECT 
    e.name,
    d.dept_name
FROM departments d
LEFT JOIN employees e 
    ON e.dept_id = d.dept_id
ORDER BY d.dept_name
'''

print("RIGHT JOIN (all departments):")
display(pd.read_sql_query(query, conn))

## 5. Multiple Table Joins

In [ ]:
# Join all three tables
query = '''
SELECT 
    e.name AS employee,
    d.dept_name,
    d.location,
    p.project_name,
    p.budget
FROM employees e
LEFT JOIN departments d 
    ON e.dept_id = d.dept_id
LEFT JOIN projects p 
    ON d.dept_id = p.dept_id
ORDER BY e.name, p.project_name
'''

pd.read_sql_query(query, conn)

## 6. Cross Join

Cartesian product of two tables.

In [ ]:
# All employee-project combinations
query = '''
SELECT 
    e.name AS employee,
    p.project_name
FROM employees e
CROSS JOIN projects p
WHERE e.dept_id IS NOT NULL
ORDER BY e.name, p.project_name
LIMIT 10
'''

pd.read_sql_query(query, conn)

## 7. Complex CTE Example

In [ ]:
# Comprehensive department analysis
query = '''
WITH 
-- Calculate department metrics
dept_metrics AS (
    SELECT 
        d.dept_id,
        d.dept_name,
        COUNT(e.emp_id) AS emp_count,
        COALESCE(AVG(e.salary), 0) 
            AS avg_salary,
        COALESCE(SUM(e.salary), 0) 
            AS total_payroll
    FROM departments d
    LEFT JOIN employees e 
        ON d.dept_id = e.dept_id
    GROUP BY d.dept_id, d.dept_name
),
-- Calculate project metrics
project_metrics AS (
    SELECT 
        dept_id,
        COUNT(*) AS project_count,
        SUM(budget) AS total_budget
    FROM projects
    GROUP BY dept_id
),
-- Combine metrics
combined AS (
    SELECT 
        dm.dept_name,
        dm.emp_count,
        dm.avg_salary,
        dm.total_payroll,
        COALESCE(pm.project_count, 0) 
            AS projects,
        COALESCE(pm.total_budget, 0) 
            AS budget
    FROM dept_metrics dm
    LEFT JOIN project_metrics pm 
        ON dm.dept_id = pm.dept_id
)
SELECT 
    dept_name,
    emp_count,
    ROUND(avg_salary, 2) AS avg_salary,
    total_payroll,
    projects,
    budget,
    CASE 
        WHEN emp_count > 0 
        THEN ROUND(
            CAST(budget AS FLOAT) / emp_count, 
            2
        )
        ELSE 0 
    END AS budget_per_employee
FROM combined
ORDER BY total_payroll DESC
'''

pd.read_sql_query(query, conn)

## Practice Exercises

### Exercise 1
Find employees who earn more than their manager.

In [ ]:
# Your code here


### Exercise 2
List departments with no employees.

In [ ]:
# Your code here


### Exercise 3
Create a CTE to find the department with 
highest average salary.

In [ ]:
# Your code here


In [ ]:
conn.close()

## Key Takeaways

✅ **CTEs** - Clean, readable complex queries  
✅ **Recursive CTEs** - Handle hierarchies  
✅ **Self joins** - Compare rows in same table  
✅ **INNER/OUTER joins** - Control matching  
✅ **Multiple joins** - Combine many tables  

**Next:** Practice Exercises →